## Линейная регрессия, Lasso и RF-регрессия в задаче по определению качества вина

In [2]:
# отключим всякие предупреждения Anaconda
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
from sklearn.metrics.regression import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.linear_model import LinearRegression, LassoCV, Lasso
from sklearn.ensemble import RandomForestRegressor

Будем работать с набором данных по качеству белого вина (репозиторий UCI). Загружаем данные.

In [4]:
data = pd.read_csv('../data/winequality-white.csv', sep=';')

In [5]:
data.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6


In [6]:
data.quality.value_counts()

6    2198
5    1457
7     880
8     175
4     163
3      20
9       5
Name: quality, dtype: int64

In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4898 entries, 0 to 4897
Data columns (total 12 columns):
fixed acidity           4898 non-null float64
volatile acidity        4898 non-null float64
citric acid             4898 non-null float64
residual sugar          4898 non-null float64
chlorides               4898 non-null float64
free sulfur dioxide     4898 non-null float64
total sulfur dioxide    4898 non-null float64
density                 4898 non-null float64
pH                      4898 non-null float64
sulphates               4898 non-null float64
alcohol                 4898 non-null float64
quality                 4898 non-null int64
dtypes: float64(11), int64(1)
memory usage: 459.3 KB


**Отделите целевой признак, разделите обучающую выборку в отношении 7:3 (30% - под оставленную выборку, пусть random_state=17) и отмасштабируйте данные с помощью StandardScaler.**

In [8]:
X = data.iloc[:, :11]
y = data['quality']

In [9]:
X_train, X_holdout, y_train, y_holdout = train_test_split(X, y, test_size=0.3, random_state=17)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_holdout_scaled = scaler.transform(X_holdout)

### Линейная регрессия

#### Обучите простую линейную регрессию.

In [10]:
linreg = LinearRegression()
linreg.fit(X_train_scaled, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

**Вопрос 1:** Каковы среднеквадратичные ошибки линейной регрессии на обучающей и отложенной выборках?

In [11]:
print("Mean squared error (train): %.3f" % mean_squared_error(y_train, linreg.predict(X_train_scaled)))
print("Mean squared error (test): %.3f" % mean_squared_error(y_holdout, linreg.predict(X_holdout_scaled)))

Mean squared error (train): 0.558
Mean squared error (test): 0.584


Посмотрите на коэффициенты модели и отранжируйте признаки по влиянию на качество вина (учтите, что большие по модулю отрицательные значения коэффициентов тоже говорят о сильном влиянии). Создайте для этого новый небольшой DataFrame.

**Вопрос 2:** Какой признак линейная регрессия считает наиболее сильно влияющим на качество вина? **-density**

In [12]:
linreg_coef = pd.DataFrame(linreg.coef_)
linreg_coef.reindex(linreg_coef.abs().sort_values(by=0, ascending=False).index)

,0
7,-0.665720
3,0.538164
1,-0.192260
8,0.150036
10,0.129533
0,0.097822
9,0.062053
5,0.042180
6,0.014304
4,0.008127


In [20]:
data.columns[7]

'density'

### Lasso-регрессия

**Обучите Lasso-регрессию с небольшим коэффициентом α=0.01 (слабая регуляризация). Пусть опять random_state=17.**

In [40]:
lasso1 = Lasso(alpha=0.01, random_state=17)
lasso1.fit(X_train_scaled, y_train)

Lasso(alpha=0.01, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=False, positive=False, precompute=False, random_state=17,
   selection='cyclic', tol=0.0001, warm_start=False)

#### Посмотрите на коэффициенты модели и отранжируйте признаки по влиянию на качество вина. Какой признак "отвалился" первым, то есть наименее важен для объяснения целевого признака в модели Lasso?

In [41]:
lasso1_coef = pd.DataFrame(lasso1.coef_)
lasso1_coef.reindex(lasso1_coef.abs().sort_values(by=0, ascending=False).index)

,0
10,0.322425
3,0.256363
7,-0.235492
1,-0.188479
8,0.067277
5,0.043088
9,0.029722
4,-0.002747
0,-0.000000
2,-0.000000


In [42]:
data.columns[0], data.columns[2], data.columns[6]

('fixed acidity', 'citric acid', 'total sulfur dioxide')

#### Теперь определите лучшее значение α в процессе кросс-валидации 5-кратной кросс-валидации. Используйте LassoCV и random_state=17.

In [45]:
alphas = np.logspace(-6, 2, 200)
lasso_cv = LassoCV(alphas=alphas, cv=5, random_state=17)
lasso_cv.fit(X_train_scaled, y_train)

LassoCV(alphas=array([1.00000e-06, 1.09699e-06, ..., 9.11589e+01, 1.00000e+02]),
    copy_X=True, cv=5, eps=0.001, fit_intercept=True, max_iter=1000,
    n_alphas=100, n_jobs=1, normalize=False, positive=False,
    precompute='auto', random_state=17, selection='cyclic', tol=0.0001,
    verbose=False)

In [46]:
lasso_cv.alpha_

0.0002833096101839324

#### Выведите коэффициенты "лучшего" Lasso в порядке убывания влияния на качество вина. 

**Вопрос 3**: Какой признак "обнулился первым" в настроенной модели LASSO?

In [48]:
lasso_cv_coef = pd.DataFrame(lasso_cv.coef_)
lasso_cv_coef.sort_values(by=0)

,0
7,-0.648161
1,-0.192049
2,-0.000000
4,0.006933
6,0.012969
5,0.042698
9,0.060939
0,0.093295
10,0.137115
8,0.146549


In [49]:
data.columns[2]

'citric acid'

**Вопрос 4**: Каковы среднеквадратичные ошибки настроенной LASSO-регрессии на обучающей и отложенной выборках?

In [50]:
print("Mean squared error (train): %.3f" % mean_squared_error(y_train, lasso_cv.predict(X_train_scaled)))
print("Mean squared error (test): %.3f" % mean_squared_error(y_holdout, lasso_cv.predict(X_holdout_scaled)))

Mean squared error (train): 0.558
Mean squared error (test): 0.583


### Случайный лес

#### Обучите случайный лес с параметрами "из коробки", фиксируя только random_state=17.

In [51]:
forest = RandomForestRegressor(random_state=17)
forest.fit(X_train_scaled, y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
           oob_score=False, random_state=17, verbose=0, warm_start=False)

**Вопрос 5**: Каковы среднеквадратичные ошибки случайного леса на обучающей выборке, на кросс-валидации (cross_val_score с scoring='neg_mean_squared_error' и остальными параметрами по умолчанию) и на отложенной выборке?

In [81]:
print("Mean squared error (train): %.3f" % mean_squared_error(y_train, forest.predict(X_train_scaled)))
print("Mean squared error (cv): %.3f" % cross_val_score(estimator=forest, X=X_train_scaled, y=y_train, 
                                                        scoring='neg_mean_squared_error').mean())
print("Mean squared error (test): %.3f" % mean_squared_error(y_holdout, forest.predict(X_holdout_scaled)))

Mean squared error (train): 0.075
Mean squared error (cv): -0.460
Mean squared error (test): 0.421


#### Настройте параметры min_samples_leaf и max_depth с помощью GridSearchCV и опять проверьте качество модели на кросс-валидации и на отложенной выборке.

In [83]:
forest_params = {'max_depth': list(range(10, 25)), 
                 'min_samples_leaf': list(range(1, 8)),
                 'max_features': list(range(6,12))}

locally_best_forest = GridSearchCV(estimator=forest, param_grid=forest_params)
locally_best_forest.fit(X_train_scaled, y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
           oob_score=False, random_state=17, verbose=0, warm_start=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'max_depth': [10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24], 'min_samples_leaf': [1, 2, 3, 4, 5, 6, 7], 'max_features': [6, 7, 8, 9, 10, 11]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [84]:
locally_best_forest.best_params_, locally_best_forest.best_score_

({'max_depth': 22, 'max_features': 7, 'min_samples_leaf': 1},
 0.42345378900629316)

**К сожалению, результаты GridSearchCV не полностью воспроизводимы (могут отличаться на разных платформах даже при фиксировании random_state). Поэтому обучите лес с параметрами max_depth=19, max_features=7, и min_samples_leaf=1 (лучшие в моем случае).**

**Вопрос 6**: Каковы среднеквадратичные ошибки настроенного случайного леса на обучающей выборке, на кросс-валидации (cross_val_score с scoring='neg_mean_squared_error') и на отложенной выборке?

In [94]:
locally_best_forest = RandomForestRegressor(max_depth=19, min_samples_leaf=1, max_features=7, random_state=17)
locally_best_forest.fit(X_train_scaled, y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=19,
           max_features=7, max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=10, n_jobs=1, oob_score=False, random_state=17,
           verbose=0, warm_start=False)

In [95]:
print("Mean squared error (cv): %.3f" % cross_val_score(estimator=locally_best_forest, X=X_train_scaled, 
                                                        y=y_train, scoring='neg_mean_squared_error').mean())
print("Mean squared error (test): %.3f" % mean_squared_error(y_holdout, locally_best_forest.predict(X_holdout_scaled)))

Mean squared error (cv): -0.457
Mean squared error (test): 0.410


**Вопрос 7**: Какой признак оказался главным в настроенной модели случайного леса?

In [96]:
rf_importance = pd.DataFrame(locally_best_forest.feature_importances_)
rf_importance.sort_values(by=0)

,0
9,0.061184
2,0.062945
0,0.064268
4,0.067982
7,0.069367
3,0.070160
6,0.071318
8,0.072806
5,0.116147
1,0.119393


In [97]:
data.columns[10]

'alcohol'